# Create a FABRIC Facility Port

This notebook shows how to use create an facility port to connect your FABRIC experiment to an external facility. 



## Import the FABlib Library

In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,17f7e488-e1b7-4ea9-b657-e69cdbb27a38
Bastion Host,bastion.fabric-testbed.net
Bastion Username,jbrassil_0034513446
Bastion Private Key File,/home/fabric/work/fabric_config/id_fabric_bastion_traffic
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


## Create the Experiment Slice

The following creates a single node with basic compute capabilities. You build a slice by creating a new slice and adding resources to the slice. After you build the slice, you must submit a request for the slice to be instantiated.   

By default, the submit function will block until the node is ready and will display the progress of your slice being built.



In [2]:
slice_name = "PRIN_facility_port"

facility_port='NetworkResearch-PRIN'
facility_port_site='PRIN'
facility_port_vlan='2000'

#Create a slice
slice = fablib.new_slice(name=slice_name)

#ex. node1 = slice.add_node(name=f"Node1", site='PRIN')
node1 = slice.add_node(name=f"Node1", 
                      site='PRIN',
                      cores=4, 
                      ram=32, 
                      disk=32, 
                      image='default_ubuntu_22')

#ex. node1_iface = node.add_component(model='NIC_ConnectX_6', name="nic10").get_interfaces()[0]
node1_iface = node1.add_component(model='NIC_Basic', name="nic10").get_interfaces()[0]

node2 = slice.add_node(name=f"Node2", 
                      site='PRIN',
                      cores=4, 
                      ram=32, 
                      disk=32, 
                      image='default_ubuntu_22')

#ex. node_iface = node.add_component(model='NIC_ConnectX_6', name="nic20").get_interfaces()[0]
node2_iface = node2.add_component(model='NIC_Basic', name="nic20").get_interfaces()[0]


facility_port = slice.add_facility_port(name=facility_port, site=facility_port_site, vlan=facility_port_vlan)
facility_port_interface = facility_port.get_interfaces()[0]

print(f"facility_port.get_site(): {facility_port.get_site()}")

# it seems facility ports needed to be l2network (i.e., l2bridge, ok for multiple sites
#net = slice.add_l2network(name=f'net_facility_port', interfaces=[node_iface,facility_port_interface])
net = slice.add_l2network(name=f'net_facility_port', interfaces=[])
net.add_interface(node1_iface)
net.add_interface(node2_iface)
net.add_interface(facility_port_interface)


#Submit the Request
slice.submit();


Retry: 11, Time: 266 sec


ID,7e786412-6000-4a52-b7db-22e3410a9ff2
Name,PRIN_facility_port
Lease Expiration (UTC),2024-11-15 15:23:41 +0000
Lease Start (UTC),2024-11-14 15:23:41 +0000
Project ID,17f7e488-e1b7-4ea9-b657-e69cdbb27a38
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
83fb5a95-bec5-479a-9138-7ac178f58d27,Node1,4,32,100,default_ubuntu_22,qcow2,prin-w2.fabric-testbed.net,PRIN,ubuntu,2620:c4:0:180:f816:3eff:fe94:56cf,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:fe94:56cf,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a5b31fb2-ed69-481a-8482-b52f4dbca568,Node2,4,32,100,default_ubuntu_22,qcow2,prin-w2.fabric-testbed.net,PRIN,ubuntu,2620:c4:0:180:f816:3eff:fed3:5422,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:fed3:5422,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
e4c17af2-6f0d-4d26-b1b9-59157ebbce3b,net_facility_port,L2,L2Bridge,PRIN,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
Node1-nic10-p1,p1,Node1,net_facility_port,100,config,,02:F8:A5:DB:47:5C,enp7s0,enp7s0,None,4
Node2-nic20-p1,p1,Node2,net_facility_port,100,config,,0A:67:13:80:B7:6D,enp7s0,enp7s0,None,4



Time to print interfaces 268 seconds


## Observe the Slice's Attributes


In [3]:
slice.show()
slice.list_nodes()
slice.list_networks()
slice.list_interfaces()


ID,7e786412-6000-4a52-b7db-22e3410a9ff2
Name,PRIN_facility_port
Lease Expiration (UTC),2024-11-15 15:23:41 +0000
Lease Start (UTC),2024-11-14 15:23:41 +0000
Project ID,17f7e488-e1b7-4ea9-b657-e69cdbb27a38
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
83fb5a95-bec5-479a-9138-7ac178f58d27,Node1,4,32,100,default_ubuntu_22,qcow2,prin-w2.fabric-testbed.net,PRIN,ubuntu,2620:c4:0:180:f816:3eff:fe94:56cf,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:fe94:56cf,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
a5b31fb2-ed69-481a-8482-b52f4dbca568,Node2,4,32,100,default_ubuntu_22,qcow2,prin-w2.fabric-testbed.net,PRIN,ubuntu,2620:c4:0:180:f816:3eff:fed3:5422,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:fed3:5422,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
e4c17af2-6f0d-4d26-b1b9-59157ebbce3b,net_facility_port,L2,L2Bridge,PRIN,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
Node1-nic10-p1,p1,Node1,net_facility_port,100,config,,02:F8:A5:DB:47:5C,enp7s0,enp7s0,None,4
Node2-nic20-p1,p1,Node2,net_facility_port,100,config,,0A:67:13:80:B7:6D,enp7s0,enp7s0,None,4


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
Node1-nic10-p1,p1,Node1,net_facility_port,100,config,,02:F8:A5:DB:47:5C,enp7s0,enp7s0,None,4
Node2-nic20-p1,p1,Node2,net_facility_port,100,config,,0A:67:13:80:B7:6D,enp7s0,enp7s0,None,4


## Run the Experiment

Most experiments will require automated configuration and execution. You can use the fablib library to execute arbitrary commands on your node. 

The following code demonstrates how to use fablib to execute a "Hello, FABRIC" bash script. The library uses the bastion and VM keys defined at the top of this notebook to jump through the bastion host and execute the script.

In [4]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

#subnet = IPv4Network("192.168.1.0/24")
subnet = IPv4Network("10.20.0.0/24")
available_ips = list(subnet)[2:]

In [5]:
node1 = slice.get_node(name=f"Node1")        
node1_iface = node1.get_interface(network_name=f'net_facility_port') 
node1_addr = available_ips.pop(99)
print(f"node1_addr: {node1_addr}")
node1_iface.ip_addr_add(addr=node1_addr, subnet=subnet)

stdout, stderr = node1.execute(f'ip addr show {node1_iface.get_os_interface()}')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_physical_os_interface_name()} up')

stdout, stderr = node1.execute(f'sudo ip link set dev {node1_iface.get_os_interface()} up')


node1_addr: 10.20.0.101
3: enp7s0: <BROADCAST,MULTICAST> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether 02:f8:a5:db:47:5c brd ff:ff:ff:ff:ff:ff
    inet 10.20.0.101/24 scope global enp7s0
       valid_lft forever preferred_lft forever


In [6]:
node2 = slice.get_node(name=f"Node2")        
node2_iface = node2.get_interface(network_name=f'net_facility_port') 
node2_addr = available_ips.pop(99)
print(f"node2_addr: {node2_addr}")
node2_iface.ip_addr_add(addr=node2_addr, subnet=subnet)

stdout, stderr = node2.execute(f'ip addr show {node2_iface.get_os_interface()}')

stdout, stderr = node2.execute(f'sudo ip link set dev {node2_iface.get_physical_os_interface_name()} up')

stdout, stderr = node2.execute(f'sudo ip link set dev {node2_iface.get_os_interface()} up')


node2_addr: 10.20.0.102
3: enp7s0: <BROADCAST,MULTICAST> mtu 1500 qdisc noop state DOWN group default qlen 1000
    link/ether 0a:67:13:80:b7:6d brd ff:ff:ff:ff:ff:ff
    inet 10.20.0.102/24 scope global enp7s0
       valid_lft forever preferred_lft forever


In [7]:
node1 = slice.get_node(name=f"Node1")     
node1_iface = node1.get_interface(network_name=f'net_facility_port') 

#stdout, stderr = node1.execute(f'ping -c 5 192.168.1.2')

#printf(f"Connect with FABRIC VM Node1")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.101')
#printf(f"Connect with Facility port switch")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.2')
#printf(f"Connect with PRIN host staas-620-3")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.3')
#printf(f"Connect with PRIN host jtb-dell-6515-2")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.4')
#printf(f"Connect with PRIN host staas-620-1 TRex interface 41")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.41')
#printf(f"Connect with PRIN host staas-620-1 Trex interface 42")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.42')

PING 10.20.0.101 (10.20.0.101) 56(84) bytes of data.
64 bytes from 10.20.0.101: icmp_seq=1 ttl=64 time=0.042 ms
64 bytes from 10.20.0.101: icmp_seq=2 ttl=64 time=0.055 ms
64 bytes from 10.20.0.101: icmp_seq=3 ttl=64 time=0.019 ms
64 bytes from 10.20.0.101: icmp_seq=4 ttl=64 time=0.026 ms
64 bytes from 10.20.0.101: icmp_seq=5 ttl=64 time=0.054 ms

--- 10.20.0.101 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4093ms
rtt min/avg/max/mdev = 0.019/0.039/0.055/0.014 ms
PING 10.20.0.2 (10.20.0.2) 56(84) bytes of data.
64 bytes from 10.20.0.2: icmp_seq=1 ttl=64 time=0.482 ms
64 bytes from 10.20.0.2: icmp_seq=2 ttl=64 time=0.263 ms
64 bytes from 10.20.0.2: icmp_seq=3 ttl=64 time=0.255 ms
64 bytes from 10.20.0.2: icmp_seq=4 ttl=64 time=0.266 ms
64 bytes from 10.20.0.2: icmp_seq=5 ttl=64 time=0.271 ms

--- 10.20.0.2 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4082ms
rtt min/avg/max/mdev = 0.255/0.307/0.482/0.087 ms
PING 10.20.0.3 (10.2

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [8]:
slice.show()

ID,7e786412-6000-4a52-b7db-22e3410a9ff2
Name,PRIN_facility_port
Lease Expiration (UTC),2024-11-15 15:23:41 +0000
Lease Start (UTC),2024-11-14 15:23:41 +0000
Project ID,17f7e488-e1b7-4ea9-b657-e69cdbb27a38
State,StableOK


ID,7e786412-6000-4a52-b7db-22e3410a9ff2
Name,PRIN_facility_port
Lease Expiration (UTC),2024-11-15 15:23:41 +0000
Lease Start (UTC),2024-11-14 15:23:41 +0000
Project ID,17f7e488-e1b7-4ea9-b657-e69cdbb27a38
State,StableOK


In [9]:
# display slice info
slice.show()

try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")   
except Exception as e:
    print(f"Exception: {e}")

ID,7e786412-6000-4a52-b7db-22e3410a9ff2
Name,PRIN_facility_port
Lease Expiration (UTC),2024-11-15 15:23:41 +0000
Lease Start (UTC),2024-11-14 15:23:41 +0000
Project ID,17f7e488-e1b7-4ea9-b657-e69cdbb27a38
State,StableOK


-----------------  ---------------------------------------------------------------------------------------------------------------------------------------
ID                 83fb5a95-bec5-479a-9138-7ac178f58d27
Name               Node1
Cores              4
RAM                32
Disk               100
Image              default_ubuntu_22
Image Type         qcow2
Host               prin-w2.fabric-testbed.net
Site               PRIN
Management IP      2620:c4:0:180:f816:3eff:fe94:56cf
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2620:c4:0:180:f816:3eff:fe94:56cf
-----------------  ---------------------------------------------------------------------------------------------------------------------------------------
-----------------  ---------------------------------------------------------------------------------------------------------------------------------------
ID      

In [10]:
# broken statement to stop execution here
JUNKprint(f"node1_addr: {node1_addr}")

NameError: name 'JUNKprint' is not defined

In [ ]:
#slice.delete()

In [ ]:
#levelup nodes and reboot
stdout, stderr = node1.execute(f"sudo apt update -qq")
stdout, stderr = node1.execute(f"sudo apt upgrade -qq  -y")
stdout, stderr = node1.execute(f"sudo apt install net-tools -y")
stdout, stderr = node1.execute(f"sudo dpkg -i ./google-chrome-stable_current_amd64.deb")
stdout, stderr = node1.execute(f"wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb")
stdout, stderr = node1.execute(f"sudo apt install x11-apps -y")
stdout, stderr = node1.execute(f"sudo apt-get install wireshark -y")
stdout, stderr = node1.execute(f"sudo dpkg-reconfigure wireshark-common")
stdout, stderr = node1.execute(f"sudo usermod -a -G wireshark $USER")
stdout, stderr = node1.execute(f"sudo reboot now")

#give node reboot some time
import time
time.sleep(30)


In [ ]:
#levelup nodes and reboot

stdout, stderr = node2.execute(f"sudo apt update -qq")
stdout, stderr = node2.execute(f"sudo apt upgrade -qq -y")
stdout, stderr = node2.execute(f"sudo apt install net-tools -y")
stdout, stderr = node2.execute(f"sudo dpkg -i ./google-chrome-stable_current_amd64.deb")
stdout, stderr = node2.execute(f"wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb")
stdout, stderr = node2.execute(f"sudo apt install x11-apps -y")
stdout, stderr = node2.execute(f"sudo apt-get install wireshark -y")
stdout, stderr = node2.execute(f"sudo dpkg-reconfigure wireshark-common")
stdout, stderr = node2.execute(f"sudo usermod -a -G wireshark $USER")
stdout, stderr = node2.execute(f"sudo reboot now")

#give node reboot some time
import time
time.sleep(30)


In [ ]:
#repair network interfaces after reboot
stdout, stderr = node1.execute(f"sudo ifconfig enp7s0 down")
stdout, stderr = node1.execute(f"sudo ip addr add 10.20.0.101/24 dev enp7s0")
stdout, stderr = node1.execute(f"sudo ifconfig enp7s0 up")

stdout, stderr = node2.execute(f"sudo ifconfig enp7s0 down")
stdout, stderr = node2.execute(f"sudo ip addr add 10.20.0.102/24 dev enp7s0")
stdout, stderr = node2.execute(f"sudo ifconfig enp7s0 up")

In [ ]:
#retest connectivity after interface restore

#stdout, stderr = node1.execute(f'ping -c 5 192.168.1.2')

#printf(f"Connect with FABRIC VM Node1")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.101')
#printf(f"Connect with FABRIC VM Node2")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.102')
#printf(f"Connect with Facility port switch")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.2')
#printf(f"Connect with PRIN host staas-620-3")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.3')
#printf(f"Connect with PRIN host jtb-dell-6515-2")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.4')
#printf(f"Connect with PRIN host staas-620-1 TRex interface 41")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.41')
#printf(f"Connect with PRIN host staas-620-1 Trex interface 42")
stdout, stderr = node1.execute(f'ping -c 5 10.20.0.42')

In [11]:
#extend slice duration
from datetime import datetime
from datetime import timezone
from datetime import timedelta

#Set slice end date to now plus 30 days
end_date = (datetime.now(timezone.utc) + timedelta(days=30)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")